## Comparison of ReAct and Plan-execute agents

In [1]:
SQL_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movies_meta.db'

PLOT_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv'
REVIEW_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv'
POSTER_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv'

In [2]:
from src.langchain.chains.movie_rag import MovieRAGChain
from src. langchain.loaders import MoviePosterDocumentLoader
from src.retrievers.visual_retriever import VisualRetriever
from src.langchain.prompts import ZERO_SHOT_QA_PROMPT

In [3]:
poster_loader = MoviePosterDocumentLoader(POSTER_PATH, max_movies=1000)   # small number since model is local
visual_docs = poster_loader.load()
visual_retriever = VisualRetriever(model_name="ViT-B/32", use_text_fusion=True, alpha=0.8)
visual_retriever.add_documents(visual_docs)

MAX_MOVIES = 500  # Limit for faster demos

# Turned off Hyde and Reranking for speed
full_chain = MovieRAGChain(
    plots_path=PLOT_PATH,
    reviews_path=REVIEW_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_hyde=True,
    hyde_model="gpt-4o-mini",
    use_reranking=True,
    reranker_cfg={'type':'llm'},
    initial_k=20, 
)
full_chain.build()

Loading posters from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv...
Created 947 poster docs.
  ✓ 947 poster documents
Loading CLIP model: ViT-B/32
✓ CLIP loaded on cpu
✓ VisualRetriever (text_fusion=True, method=weight_average)
  Alpha: 0.8 (image=0.8, text=0.2)
Encoding 947 posters with CLIP...
Encoding 947 text descriptions with CLIP...
✓ Fused embeddings using weight_average method (dim=512)
✓ Added 947 posters to index
✓ MovieRAGChain initialized
  Retriever type: custom + reranking + HyDE
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...
Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2

In [4]:
from src.agents.react import ReactAgent
from src.agents.plan_execute import PlanExecuteAgent

In [5]:
agent_react = ReactAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH)
agent_plan = PlanExecuteAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH)

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!
Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!


In [9]:
query = "What is Inception about?"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

What is Inception about?
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_Y2TasIbkPiPaUw0Pp3XBsViM)
 Call ID: call_Y2TasIbkPiPaUw0Pp3XBsViM
  Args:
    query: What is Inception about?
================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include a plot summary or details about the movie "Inception." Therefore, I cannot provide an answer regarding what "Inception" is about.
================================== Ai Message ==================================

"Inception" is a science fiction film directed by Christopher Nolan that explores the concept of shared dreams and the manipulation of the subconscious. The story follows Dom Cobb, a skilled thief who specializes in extracting secrets from people's minds while they dream. He is offered a c

In [10]:
query = "drama about love and lust with engaging red poster"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

drama about love and lust with engaging red poster
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content_and_visual (call_7E9WazQSRag2REJTjlxCz3L1)
 Call ID: call_7E9WazQSRag2REJTjlxCz3L1
  Args:
    query: drama about love and lust with engaging red poster
================================= Tool Message =================================
Name: search_movies_by_content_and_visual

Based on your request for a drama about love and lust with an engaging red poster, the following films align both textually and visually:

1. **Fifty Shades of Grey (2015)** - This film is a well-known romance that delves into themes of love and lust, particularly through its exploration of BDSM relationships. The emotional and physical dynamics between the characters are central to the plot, making it a fitting choice for your description. Additionally, the promoti

In [11]:
query = "comedy movies featuring tom hanks with good critic reviews"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

comedy movies featuring tom hanks with good critic reviews
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_aAniQfnVfVj124RyrzIeCgfc)
 Call ID: call_aAniQfnVfVj124RyrzIeCgfc
  Args:
    query: comedy movies featuring Tom Hanks with good critic reviews
================================= Tool Message =================================
Name: search_movies_by_content

Based on the provided information, there are no comedy movies featuring Tom Hanks mentioned with good critic reviews. The reviews for "Apollo 13" highlight its thrilling and dramatic nature rather than comedic elements. If you're looking for comedies specifically, "Risky Business" features Tom Cruise and is noted for its humor, but it does not include Tom Hanks. Therefore, I cannot provide a suitable answer based on the information given.
=================================

In [12]:
query = "Find dark sci-fi movies rated above 8 and explain their common themes"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Find dark sci-fi movies rated above 8 and explain their common themes
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_6skaSpn1YUrmGq2fAPmq2Wh3)
 Call ID: call_6skaSpn1YUrmGq2fAPmq2Wh3
  Args:
    question: dark sci-fi movies rated above 8
  search_movies_by_content (call_jfBbEkteoUvpUNuKABOlQUTC)
 Call ID: call_jfBbEkteoUvpUNuKABOlQUTC
  Args:
    query: common themes in dark sci-fi movies
================================= Tool Message =================================
Name: search_movies_by_content

Common themes in dark sci-fi movies include existential dread, the blurred line between humanity and technology, and the critique of corporate authority. For instance, "Alien" explores the terrifying consequences of corporate greed and the loss of human identity in a technologically advanced future. Similarly, "She Dies Tomorrow" d

In [13]:
query = "I loved Inception, Matrix, and Interstellar. Recommend 3 movies and explain why each fits my taste."

for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

I loved Inception, Matrix, and Interstellar. Recommend 3 movies and explain why each fits my taste.
================================== Ai Message ==================================
Tool Calls:
  recommend_by_similar_taste (call_XKdcUxHCEKSKqUB0OBwTqDfY)
 Call ID: call_XKdcUxHCEKSKqUB0OBwTqDfY
  Args:
    movie_titles: Inception, Matrix, Interstellar
    num_recommendations: 3
================================= Tool Message =================================
Name: recommend_by_similar_taste

{
  "warnings": "Warning: Multiple close matches found for matrix, it might also not be available in dataset. Options: ['the matrix (1999)', 'the matrix reloaded (2003)', 'the matrix revolutions (2003)']. ",
  "input_movies": [
    "Inception (2010)",
    "Interstellar (2014)"
  ],
  "method": "Item-based collaborative filtering (cosine similarity)",
  "recommendations": [
    {
      "title (year)": "Gravity (2013)",
  

In [14]:
query = "Compare Wes Anderson’s films to other colorful comedies. What defines his visual style and how do their audience ratings differ?"

for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Compare Wes Anderson’s films to other colorful comedies. What defines his visual style and how do their audience ratings differ?
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content_and_visual (call_GvY0ITopRexlUkrHDQJMvoa2)
 Call ID: call_GvY0ITopRexlUkrHDQJMvoa2
  Args:
    query: Wes Anderson films visual style and colorful comedies
  search_movies_by_metadata (call_P9RlEIyVe71b4tRWa0hDt5le)
 Call ID: call_P9RlEIyVe71b4tRWa0hDt5le
  Args:
    question: Compare audience ratings of Wes Anderson films to other colorful comedies
================================= Tool Message =================================
Name: search_movies_by_metadata

          film_category  average_audience_rating
Other Colorful Comedies                60.293601
     Wes Anderson Films                84.888889
================================== Ai Message ==========

In [6]:
query = "Between Christopher Nolan and Denis Villeneuve, whose recent sci-fi films are rated higher, and what themes do they share?"

for agent in [agent_plan]:    # react agent failed on this one
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Between Christopher Nolan and Denis Villeneuve, whose recent sci-fi films are rated higher, and what themes do they share?

Plan created (3 steps):
  1. Use **search_movies_by_metadata** to find the recent sci-fi films by Christopher Nolan and Denis Villeneuve, including their ratings. This will help identify which director's films are rated higher.
  2. Use **search_movies_by_content** to analyze the themes present in the recent sci-fi films of both Christopher Nolan and Denis Villeneuve. This will provide insight into the thematic similarities and differences between their works.
  3. Synthesize results to compare the ratings of the recent sci-fi films from both directors and identify the shared themes between them.

Executing step 1/3: 1. Use **search_movies_by_metadata** to find the recent sci-fi films by Christopher Nolan and Denis Villeneuve, including their ratings. This will help identify which di

In [7]:
query = "Analyze how critic sentiment towards superhero movies has changed from the 2000s to the 2020s."

for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Analyze how critic sentiment towards superhero movies has changed from the 2000s to the 2020s.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_r7sqbu51OY9fC3c4fcWBSlkT)
 Call ID: call_r7sqbu51OY9fC3c4fcWBSlkT
  Args:
    question: Average critic ratings of superhero movies from the 2000s
  search_movies_by_metadata (call_2FR709cIe3gvYaa98FhFQWnY)
 Call ID: call_2FR709cIe3gvYaa98FhFQWnY
  Args:
    question: Average critic ratings of superhero movies from the 2020s
================================= Tool Message =================================
Name: search_movies_by_metadata

average_critic_rating
                 None
================================= Tool Message =================================
Name: search_movies_by_metadata

average_critic_rating
                 None
================================== Ai Message ========

#### Testing and comparing web-search tool for both

In [8]:
agent_react = ReactAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)
agent_plan = PlanExecuteAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!
Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!


In [9]:
query = "When is Severence new season going to be released?"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

When is Severence new season going to be released?
================================== Ai Message ==================================
Tool Calls:
  search_web_for_movies (call_u1k2B5xbm6bOH4vGKOAUg2Y2)
 Call ID: call_u1k2B5xbm6bOH4vGKOAUg2Y2
  Args:
    query: Severance new season release date
================================= Tool Message =================================
Name: search_web_for_movies

As of now, there is no official release date for *Severance* Season 3. Filming is expected to start in 2026, which means the new season will not premiere in that year. While the show has been confirmed for a third season, fans will likely have to wait some time before it is available on Apple TV+.
================================== Ai Message ==================================

As of now, there is no official release date for *Severance* Season 3. Filming is expected to start in 2026, so the new season will no

In [10]:
query = "What is one of the best rated movies of 2025? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies."
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

What is one of the best rated movies of 2025? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_iVo7KLuGAlgtc8Hovdf1FRRg)
 Call ID: call_iVo7KLuGAlgtc8Hovdf1FRRg
  Args:
    question: Best rated movies of 2025
================================= Tool Message =================================
Name: search_movies_by_metadata

No movies found matching: Best rated movies of 2025. Tried query: SQL: SELECT movie_title, imdb_rating, tomatometer_rating, audience_rating
FROM movies
WHERE release_year = 2025
ORDER BY imdb_rating DESC, tomatometer_rating DESC, audience_rating DESC;
================================== Ai Message ==================================
Tool Calls:
  search_web_fo

##### We can see that plan-execute can fail if we don't allow replanning!

## Plan-execute agent with replanning, retry, fallback capability

In [11]:
from src.agents.adaptive_plan_execute import AdaptivePlanExecuteAgent

In [12]:
agent_plan_ad = AdaptivePlanExecuteAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!


In [13]:
query = "what is the ending of inception?"
_ = agent_plan_ad.query(query, verbose=True)

================================ Human Message =================================

what is the ending of inception?

Plan created (2 steps):
  1. Use **search_movies_by_content** to find a summary or analysis of the movie "Inception" that includes details about its ending.
  2. Synthesize results to answer the question about the ending of "Inception."

Executing step 1/2: 1. Use **search_movies_by_content** to find a summary or analysis of the movie "Inception" that includes details about its ending.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_wcF6Y1sJYjdmqJI4qg0nQxXP)
 Call ID: call_wcF6Y1sJYjdmqJI4qg0nQxXP
  Args:
    query: Inception ending explained
================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include any details about the ending of "Inception." Therefore, I cannot explain the ending of that film.
 Ste

In [14]:
query = "What is one of the most well received horror movies of 2025 in terms of imdb? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies."
_ = agent_plan_ad.query(query, verbose=True)

================================ Human Message =================================

What is one of the most well received horror movies of 2025 in terms of imdb? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies.

Plan created (5 steps):
  1. Use **search_web_for_movies** to find the most well-received horror movies of 2025 in terms of IMDb ratings. This will provide the title of a highly rated horror movie released in that year.
  2. Use **search_movies_by_metadata** to find the main actor/actress's name for the identified horror movie. This will give us the key cast member associated with the film.
  3. Use **search_movies_by_content** to find out what type of movies the main actor/actress has acted in. This will help us understand their filmography and the genres they typically engage with.
  4. Use **recommend_by_similar_taste** to find other movies that you might like if you're 

In [16]:
query = "I love normal people. Can you tell me who the main actor was and what type of movies he's acted in? If I like that movie what else might I be interested in?"
_ = agent_plan_ad.query(query, verbose=True)

================================ Human Message =================================

I love normal people. Can you tell me who the main actor was and what type of movies he's acted in? If I like that movie what else might I be interested in?

Plan created (3 steps):
  1. Use **search_movies_by_content** to find out who the main actor in "Normal People" is and gather information about the types of movies he has acted in.
  2. Use **recommend_by_similar_taste** to find movies that are similar to "Normal People" based on critic rating patterns, which will help identify what else the user might be interested in.
  3. Synthesize results to provide the user with the main actor's name, the types of movies he has acted in, and a list of recommended movies based on their interest in "Normal People."

Executing step 1/3: 1. Use **search_movies_by_content** to find out who the main actor in "Normal People" is and gather information about the types of movies he has acted in.
=========================

## Adding human in the loop after fallback fails (special evaluator for fallback results)

#### Also, adding an llm to summarize web search tool's results!

In [4]:
from src.agents.hitl_adaptive_plan_execute import AdaptHITLPlanExecAgent

In [21]:
agent_plan_ad_hitl = AdaptHITLPlanExecAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!


In [10]:
query = "What is one of the most well received horror movies of 2025 in terms of imdb? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies."
_ = agent_plan_ad_hitl.query(query, verbose=True)

================================ Human Message =================================

What is one of the most well received horror movies of 2025 in terms of imdb? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies.

Plan created (5 steps):
  1. Use **search_web_for_movies** to find the most well-received horror movies of 2025 in terms of IMDb ratings. This will provide the title of a highly rated horror movie released in that year.
  2. Use **search_movies_by_metadata** to find the main actor/actress's name for the identified horror movie. This will give us the key cast information needed for the next steps.
  3. Use **search_movies_by_content** to find out what type of movies the main actor/actress has acted in. This will help us understand their filmography and the genres they typically engage with.
  4. Use **recommend_by_similar_taste** to find other movies that you might like if y

1=Refine query, 2=Skip, 3=Continue with this result, 4=Replan, 5=Stop:  3



 Continuing with current results as requested.

Executing step 3/5: 3. Use **search_movies_by_content** to find out what type of movies the main actor/actress has acted in. This will help us understand their filmography and the genres they typically engage with.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_GOKR5TftawdXMI85ccupao0t)
 Call ID: call_GOKR5TftawdXMI85ccupao0t
  Args:
    query: Aaron Taylor-Johnson filmography
  search_movies_by_content (call_vzrPZxvS4Ba62KXgQ1KW8Jyv)
 Call ID: call_vzrPZxvS4Ba62KXgQ1KW8Jyv
  Args:
    query: Jodie Comer filmography
================================= Tool Message =================================
Name: search_movies_by_content

Aaron Taylor-Johnson appears in the following films based on the provided information:

1. **Anna Karenina** (2012) - Directed by Joe Wright, this drama and romance film features Taylor-Johnson among a cast that includes Keira Knightley 

1=Refine query, 2=Skip, 3=Continue with this result, 4=Replan, 5=Stop:  3



 Continuing with current results as requested.

Executing step 4/5: 4. Use **recommend_by_similar_taste** to find other movies that you might like if you're a fan of the main actor/actress's movies. This will provide tailored recommendations based on their previous works.
================================== Ai Message ==================================
Tool Calls:
  recommend_by_similar_taste (call_ujzuOsNH5LGnzfMVB1JATfTm)
 Call ID: call_ujzuOsNH5LGnzfMVB1JATfTm
  Args:
    movie_titles: Nowhere Boy, Kick-Ass, Godzilla, Bullet Train
    num_recommendations: 5
  recommend_by_similar_taste (call_ZV33YenywSPcxB7wYYjbDTQt)
 Call ID: call_ZV33YenywSPcxB7wYYjbDTQt
  Args:
    movie_titles: Free Guy, The Last Duel, Killing Eve
    num_recommendations: 5
================================= Tool Message =================================
Name: recommend_by_similar_taste

{
  "warnings": "Warning: Multiple movies found for godzilla. Options: ['godzilla (1998)', 'godzilla (2014)']. Warning: Multipl

1=Refine query, 2=Skip, 3=Continue with this result, 4=Replan, 5=Stop:  1
How should we refine the query?  Search web for movies similar to Kick-Ass, Godzilla, Bullet Train; and Free Guy, The Last Duel, Killing Eve



 Continuing with refined step as requested.

Executing step 4/5: Search web for movies similar to Kick-Ass, Godzilla, Bullet Train; and Free Guy, The Last Duel, Killing Eve
================================== Ai Message ==================================
Tool Calls:
  recommend_by_similar_taste (call_dBkFm9mNNEwDJ7zjdsL0jScM)
 Call ID: call_dBkFm9mNNEwDJ7zjdsL0jScM
  Args:
    movie_titles: Kick-Ass, Godzilla, Bullet Train
  recommend_by_similar_taste (call_P5KUXmggqED6rKSIh3JqJmGa)
 Call ID: call_P5KUXmggqED6rKSIh3JqJmGa
  Args:
    movie_titles: Free Guy, The Last Duel, Killing Eve
================================= Tool Message =================================
Name: recommend_by_similar_taste

{
  "warnings": "Warning: Multiple movies found for godzilla. Options: ['godzilla (1998)', 'godzilla (2014)']. Warning: Multiple close matches found for bullet train, it might also not be available in dataset. Options: ['the midnight meat train (2008)', 'mystery train (1989)', 'bullet to the h

In [11]:
query = "I love normal people. Can you tell me who the main actor was and what type of movies he's acted in? If I like that movie what else might I be interested in?"
_ = agent_plan_ad_hitl.query(query, verbose=True)

================================ Human Message =================================

I love normal people. Can you tell me who the main actor was and what type of movies he's acted in? If I like that movie what else might I be interested in?

Plan created (3 steps):
  1. Use **search_movies_by_content** to find out who the main actor in "Normal People" is and gather information about the types of movies he has acted in.
  2. Use **recommend_by_similar_taste** to find movies that share similar critic rating patterns with "Normal People" to suggest what else the user might be interested in.
  3. Synthesize results to provide the user with the main actor's name, a brief overview of his filmography, and a list of recommended movies based on their interest in "Normal People."

Executing step 1/3: 1. Use **search_movies_by_content** to find out who the main actor in "Normal People" is and gather information about the types of movies he has acted in.
================================== Ai Message

I Love this answer! Adding summarizer to web search tool helped a LOT! And human in the loop is very fun to use

In [12]:
query = "I love the movie where the girl was sick. What was the name? I remember the actress was blond and passed away in the end"
_ = agent_plan_ad_hitl.query(query, verbose=True)

================================ Human Message =================================

I love the movie where the girl was sick. What was the name? I remember the actress was blond and passed away in the end

Plan created (3 steps):
  1. Use **search_movies_by_content** to find movies that feature a plot involving a girl who is sick and has blonde hair, and where the character passes away at the end. This will help identify potential movies that match the description provided by the user.
  2. Synthesize results to identify the most likely movie based on the information retrieved, focusing on the details about the character and the actress.
  3. Present the identified movie title to the user, confirming if it matches their memory.

Executing step 1/3: 1. Use **search_movies_by_content** to find movies that feature a plot involving a girl who is sick and has blonde hair, and where the character passes away at the end. This will help identify potential movies that match the description provid

In [15]:
query = "List the top 5 movies based on the reviews, and from each list all the other movies the main character has played in still sorted by those movies' reviews. Include the rating for every movie you list."
_ = agent_plan_ad_hitl.query(query, verbose=True)

================================ Human Message =================================

List the top 5 movies based on the reviews, and from each list all the other movies the main character has played in still sorted by those movies' reviews. Include the rating for every movie you list.

Plan created (4 steps):
  1. Use **search_movies_by_metadata** to retrieve the top 5 movies based on reviews. This will provide a list of the highest-rated movies along with their ratings.
  2. For each of the top 5 movies identified in step 1, use **search_movies_by_content** to find out who the main character is and list all the other movies that actor has played in. This will help gather the filmography of the main character for each of the top movies.
  3. For each movie from the filmography obtained in step 2, use **search_movies_by_metadata** again to get the ratings for those movies. This will ensure that all movies listed are sorted by their reviews.
  4. Synthesize results to compile a comprehensiv

1=Refine query, 2=Skip, 3=Continue with this result, 4=Replan, 5=Stop:  3



 Continuing with current results as requested.

Executing step 3/4: 3. For each movie from the filmography obtained in step 2, use **search_movies_by_metadata** again to get the ratings for those movies. This will ensure that all movies listed are sorted by their reviews.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_8cd00BUaAz3T2vBT2qaKOeVO)
 Call ID: call_8cd00BUaAz3T2vBT2qaKOeVO
  Args:
    question: List all movies starring Henry Fonda.
  search_movies_by_metadata (call_Q4WCuZUcUzhSduQFL6AbkhWc)
 Call ID: call_Q4WCuZUcUzhSduQFL6AbkhWc
  Args:
    question: List all movies starring Takashi Shimura.
  search_movies_by_metadata (call_LLvKZ5W7TPTLrTQ9kZg31joe)
 Call ID: call_LLvKZ5W7TPTLrTQ9kZg31joe
  Args:
    question: List all movies starring Charlie Chaplin.
  search_movies_by_metadata (call_q4PdDP2wkU9qP3voUsriaVkW)
 Call ID: call_q4PdDP2wkU9qP3voUsriaVkW
  Args:
    question: List all movies starrin

1=Refine query, 2=Skip, 3=Continue with this result, 4=Replan, 5=Stop:  3



 Continuing with current results as requested.

Synthesizing results...
================================== Ai Message ==================================

Based on the reviews, here are the top 5 movies along with their ratings and the filmography of the main characters sorted by their ratings:

### Top 5 Movies Based on Reviews

1. **12 Angry Men (Twelve Angry Men)**
   - **Rating:** 100% (Tomatometer)
   - **Main Character:** Juror 8, portrayed by **Henry Fonda**
   - **Henry Fonda's Notable Films:**
     1. **On Golden Pond** (1981) - Rating: 92%
     2. **Spencer's Mountain** (1963) - Rating: 80%
     3. **The Oldest Living Graduate** (1980) - Rating: 75%
     4. **The Deputy** (1959) - Rating: 70%

2. **Seven Samurai (Shichinin no Samurai)**
   - **Rating:** 100% (Tomatometer)
   - **Main Character:** Kambei Shimada, portrayed by **Takashi Shimura**
   - **Takashi Shimura's Notable Films:**
     1. **Ikiru** (1952) - Rating: 98%
     2. **Rashômon** (1950) - Rating: 97%
     3. **

In [22]:
query = "Between Christopher Nolan and Denis Villeneuve, whose recent sci-fi films are rated higher, and what themes do they share?"
_ = agent_plan_ad_hitl.query(query, verbose=True)

================================ Human Message =================================

Between Christopher Nolan and Denis Villeneuve, whose recent sci-fi films are rated higher, and what themes do they share?

Plan created (3 steps):
  1. Use **search_movies_by_metadata** to find the recent sci-fi films directed by Christopher Nolan and Denis Villeneuve, and retrieve their ratings. This will help us compare the ratings of their recent works.
  2. Use **search_movies_by_content** to identify the themes present in the recent sci-fi films of both Christopher Nolan and Denis Villeneuve. This will allow us to analyze the thematic similarities and differences between their films.
  3. Synthesize results to answer the question regarding which director's recent sci-fi films are rated higher and what themes they share.

Executing step 1/3: 1. Use **search_movies_by_metadata** to find the recent sci-fi films directed by Christopher Nolan and Denis Villeneuve, and retrieve their ratings. This will he

1=Refine query, 2=Skip step, 3=Continue with this result, 4=Replan, 5=Stop:  3



 Continuing with current results as requested.

Executing step 2/3: 2. Use **search_movies_by_content** to identify the themes present in the recent sci-fi films of both Christopher Nolan and Denis Villeneuve. This will allow us to analyze the thematic similarities and differences between their films.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_8Hdh59eBhJdv6DDhONWZfr9l)
 Call ID: call_8Hdh59eBhJdv6DDhONWZfr9l
  Args:
    query: Inception themes
  search_movies_by_content (call_xAc2yrIpaTtdlIy1e8axpgl0)
 Call ID: call_xAc2yrIpaTtdlIy1e8axpgl0
  Args:
    query: Interstellar themes
  search_movies_by_content (call_fU5Im2nrEO9aKvxAQrYUIHRf)
 Call ID: call_fU5Im2nrEO9aKvxAQrYUIHRf
  Args:
    query: Tenet themes
  search_movies_by_content (call_sBbTr4urAIcTGRG3CcIELZJg)
 Call ID: call_sBbTr4urAIcTGRG3CcIELZJg
  Args:
    query: Arrival themes
  search_movies_by_content (call_NhEKJazsePgVbLMptckSR6Pz)
 Call I